In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from generator import DataGenerator
PROJECT_NAME = "LPRnet_keras"
MODEL_NAME = "depthwise_model_randomchars_perspective_multiple_fonts"

IMAGE_SHAPE = [94,24]
CHARS = "ABCDEFGHIJKLMNPQRSTUVWXYZ0123456789" # exclude I, O
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
DECODE_DICT = {i:char for i, char in enumerate(CHARS)}
NUM_CLASS = len(CHARS)+1

In [2]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    
    return loss

In [3]:
class small_basic_block(keras.layers.Layer):

    def __init__(self,out_channels,name=None,**kwargs):
        super().__init__(**kwargs)
        out_div4=int(out_channels/4)
        self.main_layers = [
            keras.layers.SeparableConv2D(out_div4,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_div4,(3,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_div4,(1,3),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_channels,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]  
    
    def call(self,input):
        x = input
        for layer in self.main_layers:
            x = layer(x)
        return x

In [4]:
#test this later

class global_context(keras.layers.Layer):
    def __init__(self,kernel_size,stride,**kwargs):
        super().__init__(**kwargs)
        self.ksize = kernel_size
        self.stride = stride

    def call(self, input):
        x = input 
        avg_pool = keras.layers.AveragePooling2D(pool_size=self.ksize,strides=self.stride,padding='same')(x)
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(avg_pool)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        out = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([avg_pool , sqm])
        #out = keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(avg_pool)
        return out

    def get_config(self):
        return {
            "kernel_size": self.ksize,
            "stride": self.stride,
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [5]:
class LPRnet(keras.Model):
    def __init__(self, input_shape=(24,94,3), **kwargs):
        super(LPRnet, self).__init__(**kwargs)
        self.input_layer = keras.layers.Input(input_shape)
        self.cnn_layers= [
            keras.layers.SeparableConv2D(64,kernel_size = (3,3),strides=1,padding='same',name='main_conv1',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(name='BN1'),
            keras.layers.ReLU(name='RELU1'),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,1),name='maxpool2d_1',padding='same'),
            small_basic_block(128),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_2',padding='same'),
            small_basic_block(256),
            small_basic_block(256),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_3',padding='same'),
            keras.layers.Dropout(0.5),
            keras.layers.SeparableConv2D(256,(4,1),strides=1,padding='same',name='main_conv2',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Dropout(0.5),
            keras.layers.SeparableConv2D(NUM_CLASS,(1,13),padding='same',name='main_conv3',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),  
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]
        self.out_layers = [
            keras.layers.SeparableConv2D(NUM_CLASS,kernel_size=(1,1),strides=(1,1),padding='same',name='conv_out',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
        ]
        self.out = self.call(self.input_layer)

    def call(self,inputs,training=False):
        x = inputs
        layer_outputs = []
        for layer in self.cnn_layers:
            x = layer(x)
            layer_outputs.append(x)
        scale1 = global_context((1,4),(1,4))(layer_outputs[0])
        scale2 = global_context((1,4),(1,4))(layer_outputs[4])
        scale3 = global_context((1,2),(1,2))(layer_outputs[6])
        scale5 = global_context((1,2),(1,2))(layer_outputs[7])
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(x)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        scale4 = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([x , sqm])
        gc_concat = keras.layers.Lambda(lambda x: tf.concat([x[0], x[1], x[2], x[3], x[4]],3))([scale1, scale2, scale3, scale5,scale4])
        for layer in self.out_layers:
            gc_concat = layer(gc_concat)
        logits = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x[0],axis=1))([gc_concat])
        logits = keras.layers.Softmax()(logits)
        return logits

In [6]:
import glob
import cv2

real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\valid\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')

data = []
labels = []
val_data = []
val_labels = []

for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))/256
    data.append(image)
    labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

for file in real_images_val:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))
    val_data.append(image)
    val_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

training_set = np.array(data,dtype=np.float32)
training_labels = np.array(labels)
ragged = tf.ragged.constant(training_labels).to_tensor()
real_dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).batch(64)


C:\Users\carlos\AppData\Local\Temp/ipykernel_24876/2036217571.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(labels)


In [7]:
if os.path.exists(MODEL_NAME):
    model = keras.models.load_model(
        MODEL_NAME, custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
    )
else:
    model = LPRnet()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss =CTCLoss)
    model.build((1,24,94,3))

In [8]:
from gen_plates_keras import *
gen = ImageGenerator()
def generate_dataset(num = 100):
    data, labels = gen.generate_images(num)
    gen_labels = []
    for label in labels:
        gen_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
    pics =np.array(data)
    labels = np.array(labels)
    training_set = np.array(pics,dtype=np.float32)
    training_labels = np.array(gen_labels)
    ragged = tf.ragged.constant(training_labels).to_tensor()
    dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).batch(64)
    return dataset
test_dataset = generate_dataset()

C:\Users\carlos\AppData\Local\Temp/ipykernel_24876/2355684483.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


In [9]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project=MODEL_NAME, entity="clsandoval")
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 400,
  "batch_size": 64
}

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: clsandoval (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [11]:
generate = DataGenerator()
check = tf.keras.callbacks.ModelCheckpoint(
    './trained_models/{}'.format(MODEL_NAME),
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq=500,
    options=None,
)
model.fit(test_dataset)
model.fit_generator(generator=generate,validation_data=real_dataset,validation_steps=3,epochs=11,steps_per_epoch=50,callbacks=[check])


2/2 [==============================] - 1s 194ms/step - loss: 60.0428
Epoch 1/11


c:\Users\carlos\Desktop\cs\ml-sandbox\ANPR\LPRnet-keras\generator.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


50/50 [==============================] - 16s 280ms/step - loss: 27.9636 - val_loss: 100.2028
Epoch 2/11
50/50 [==============================] - 13s 254ms/step - loss: 22.8943 - val_loss: 100.2028
Epoch 3/11
50/50 [==============================] - 13s 254ms/step - loss: 22.5133 - val_loss: 100.2007
Epoch 4/11
50/50 [==============================] - 20s 408ms/step - loss: 22.2442 - val_loss: 100.0725
Epoch 5/11
50/50 [==============================] - 16s 322ms/step - loss: 22.1409 - val_loss: 99.1084
Epoch 6/11
50/50 [==============================] - 25s 489ms/step - loss: 22.1597 - val_loss: 97.3229
Epoch 7/11
50/50 [==============================] - 12s 247ms/step - loss: 21.9344 - val_loss: 88.6508
Epoch 8/11
50/50 [==============================] - 13s 262ms/step - loss: 22.0578 - val_loss: 71.5353
Epoch 9/11
50/50 [==============================] - 12s 247ms/step - loss: 21.9304 - val_loss: 57.6076
Epoch 10/11
50/50 [==============================] - ETA: 0s - loss: 21.6736

INFO:tensorflow:Assets written to: ./trained_models\depthwise_model_randomchars_perspective_multiple_fonts\assets


INFO:tensorflow:Assets written to: ./trained_models\depthwise_model_randomchars_perspective_multiple_fonts\assets


50/50 [==============================] - 18s 353ms/step - loss: 21.6736 - val_loss: 53.2220
Epoch 11/11


c:\Users\carlos\Desktop\cs\ml-sandbox\ANPR\LPRnet-keras\generator.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


50/50 [==============================] - 16s 322ms/step - loss: 21.3239 - val_loss: 44.1192


In [12]:

TFLITE_PATH = 'tflite_models'
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("./{}/{}.tflite".format(TFLITE_PATH,MODEL_NAME), 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmpmwi4e7jm\assets


INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmpmwi4e7jm\assets


In [13]:
interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\{}.tflite".format(MODEL_NAME))
#interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\keras_lprnet_separable.tflite")
import numpy as np
import time 
import glob
import cv2

interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')
start = time.time()
ctr = 0
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = np.expand_dims(test_image,axis=0)
    test_image = test_image.astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    decoded = keras.backend.ctc_decode(output_data,(24,),greedy=False)
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text:
        ctr += 1
    print(text, " "+ label)
print(time.time()-start)
print(ctr,len(real_images))

A  AAC1586
TA  459DLY
EA  AAO5592
EA  APA3772
A  DAA4626
A  DAF9190
A  DAJ1083
A  DAL3300
A  DCP7079
A  NBU4828
A  NCG1221
9A  NCH8029
A  NCM9415
A  NEH1815
EA  NFX7563
A  NGS2592
A  AAJ5201
A  AAP1274
TA  AAW1583
TZA  ABA8615
A  AEA9483
ZA  AEA9512
A  ARA6828
A  ASA8680
TA  CAQ6650
TZA  CPY711
A  DAG3746
TA  DAH3150
A  DAK1304
A  DAL7407
TA  DAO6200
A  DAY1428
A  DCP5431
EA  DCP7323
A  NA36503
A  NAE1322
EA  NAE2008
EA  NAJ8740
A  NBA4750
TA  NBK6857
A  NBP3395
A  NBR9407
A  NC04583
A  NCD4761
A  NCM8042
A  NCM9147
A  NDE9713
A  NDF2712
A  NDJ3111
TA  NDS1024
TA  NDV1148
EA  NEA4294
TA  NED2300
A  NED3275
A  NEH4482
TA  NGJ4631
A  TSN402
A  153NDE
EA  ABR5310
A  CAB1069
TA  NA02641
UA  NBC2309
TA  NBF8676
A  NDT7127
A  NE26216
A  ABE2313
9A  ABH1544
TA  NBI5456
TA  NBT2564
TA  NDT8216
EA  NEB8274
TA  NFT6109
TA  401DJL
TA  ADA5892
ZA  APA7080
A  CDZ5495
TA  DAL8625
TA  DRL923
TA  MJZ168
A  NAT6342
TA  NCC1100
TA  NDC2372
TA  NEA5661
TA  510TPD
A  AAM4901
A  AAQ8870
A  AXA3085
EA  DAA7

In [5]:
MODEL_PATH = 'trained_models'
TFLITE_PATH = 'tflite_models'
model = keras.models.load_model(
    os.path.join(MODEL_PATH, MODEL_NAME), custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
)

In [7]:
import glob
import cv2
real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\*\\*\\*.png')
ctr = 0
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    image = cv2.imread(file).astype('float32')
    test_image = cv2.resize(image,(94,24))/256
    test_image = np.expand_dims(test_image,axis=0)
    preds = model.predict(test_image) 
    decoded = tf.keras.backend.ctc_decode(preds,(24,))
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text:
        ctr += 1
    print(text," "+ label)
print(ctr)
print(len(real_images))

587S05LAA  07303
7301Q85B125  1303
130340843  1303
566TPEAA  566TPE
35TNMSA  57
EE800NGMAA  59
A103573AA  AA03573
AAK17053A  AAK7053
AAE4G73A  AAL4673
ATG3TN5A  ACG3105
AB352S3A  AD35293
DAL5017A  DAL5017
DAMA00  DAM7900
DC18721  DC18721
G1GT5EA  G2G725
WAF6171  NAF6171
NBK7446  NBK7446
MG5725ZA  NC57254
NCG109Z  NCG1092
NCZ5306  NCZ5306
SS83253GA  NDB3253
NE70G3GA  NE70636
NE70G3GA  NE84727
NEJ9900  NEJ9900
MF2Y85A  NH
PYY50TA  PYV501
WSNMBAA  TJX496
VW50T54A  WSO154
ZUVZ5TAA  ZJV257
TG30QRDR955A  1303
943N0XA  943NOX
UDAE3011  DAE3011
NAH7401  NAH7401
W8G1384  NBG1384
NBV2323U  NBV2323
NC16514  NCI6514
FNCJ9396  NCJ9396
LNCU375B  NCU3758
ND11397  NDI1397
4SQT580E7AA  NDT5180
NDU6721A  NDU6721
ZNET1791A  NET1793
NGG6277A  NGG6277
TTXSKQTAA  TYX907
0GEB09AA  UGF809
BET339AA  WET339
659RZUFA  XSC203
Y5G054  YS6054
AAH7062  AAH7062
AAZ2G55  AAZ2655
ABG5953  ABG5953
ACZEQBAA  ACZ4349
2P1BT75A  APA8716
NAD1140  NAD1140
N86M27805  NBQ2703
TNCY8S40  NCY8340
NDG3354  NDG3354
UU2B45A  UIT846
0

In [24]:
val_training_set = np.array(val_data,dtype=np.float32)
val_training_labels = np.array(val_labels)
val_ragged = tf.ragged.constant(val_training_labels).to_tensor()
val_dataset = tf.data.Dataset.from_tensor_slices((val_training_set,val_ragged)).shuffle(640).batch(64)

C:\Users\carlos\AppData\Local\Temp/ipykernel_21744/3660101917.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_training_labels = np.array(val_labels)
